In [39]:
import pandas as pd
import datetime
from datetime import timedelta
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

fed_path = '/Users/chenshikai/Downloads/Fed events v3.csv'
trade_path = '/Users/chenshikai/Desktop/Trade disputes events v2.csv'

fed_dataframe = pd.read_csv(fed_path)
trade_dataframe = pd.read_csv(trade_path)


In [40]:
trade_dataframe.head()

,Event,Year,Month,Day,Company Name,Ticker,Market,Industry,Country
0,US implemented universal tariffs on solar pane...,2018,2,7,DowDuPont Inc.,DWDP,^DJI,Basic Materials (Chemicals),U.S.
1,US implemented universal tariffs on solar pane...,2018,2,7,3M Co.,MMM,^DJI,Business/Consumer Services,U.S.
2,US implemented universal tariffs on solar pane...,2018,2,7,International Business Machines Corp.,IBM,^DJI,Business/Consumer Services,U.S.
3,US implemented universal tariffs on solar pane...,2018,2,7,Exxon Mobil Corp.,XOM,^DJI,Energy,U.S.
4,US implemented universal tariffs on solar pane...,2018,2,7,Chevron Corp.,CVX,^DJI,Energy,U.S.


In [41]:
# Trade data processing
trade_dataframe = pd.DataFrame(pd.read_csv('/Users/chenshikai/Desktop/Trade disputes events v2.csv'))
trade_event_name = trade_dataframe['Event']
trade_company = trade_dataframe['Ticker']
trade_company_name = trade_dataframe['Company Name']
trade_market = trade_dataframe['Market'] #240
trade_industry = trade_dataframe['Industry']
trade_year = trade_dataframe['Year']
trade_month = trade_dataframe['Month']
trade_day = trade_dataframe['Day']
# trade_country = trade_dataframe['Country']

trade_dic = {}
for i in range(0, len(trade_market)):
    trade_dic[trade_company[i]] = trade_market[i]

# Add an "EventDate" column to the main dataframe
trade_eventdate = []
for i in range(0, len(trade_dataframe)):
    trade_eventdate.append(datetime.datetime(trade_year[i], trade_month[i], trade_day[i]))
trade_dataframe['EventDate'] = trade_eventdate

# trade_dataframe
trade_inter_days = []
trade_uniquedays = sorted(trade_dataframe['EventDate'].unique()) # 8 unique days
for i in range(0, len(trade_uniquedays)-1):
    trade_inter_days.append(int((trade_uniquedays[i+1] - trade_uniquedays[i])/np.timedelta64(1,'D')))
trade_sample_length = min(trade_inter_days) # 84, inter_days:  [364, 91, 91, 182, 98, 84, 105]
# fed_sample_length
trade_eventdate_set = trade_dataframe['EventDate'].unique()
# len(trade_eventdate_set)

In [42]:
trade_sample_length

4

In [44]:



# match stock with market (either DJI or NASDAQ)
trade_dic = {}
for i in range(0, len(trade_market)):
    trade_dic[trade_company[i]] = trade_market[i]

#Add an "EventDate" column to the main dataframe
trade_eventdate = []
for i in range(0, len(trade_dataframe)):
    trade_eventdate.append(datetime.datetime(trade_year[i], trade_month[i], trade_day[i]))
trade_dataframe['EventDate'] = trade_eventdate

trade_inter_days = []
trade_uniquedays = sorted(trade_dataframe['EventDate'].unique()) # 8 unique days
for i in range(0, len(trade_uniquedays)-1):
    trade_inter_days.append(int((trade_uniquedays[i+1] -trade_uniquedays[i])/np.timedelta64(1,'D')))
trade_sample_length = min(trade_inter_days) # 84, inter_days:  [364, 91, 91, 182, 98, 84, 105]


#### Regression #####

trade_reg_result= []

# inter_days
trade_eventdate_set = trade_dataframe['EventDate'].unique()
trade_subdate_cumulative_abnormal_return = pd.DataFrame()

for date in trade_eventdate_set:
    trade_subevent_df = trade_dataframe[trade_dataframe.EventDate == date]
    trade_start_date = date - np.timedelta64(trade_sample_length, 'D')
    trade_end_date = date + np.timedelta64(trade_sample_length, 'D')
    
    trade_subevent_company_list = trade_subevent_df['Ticker'].unique()
    trade_subevent_cumulative_abnormal_return = pd.DataFrame() # again?

    for company in trade_subevent_company_list:
        trade_price = web.DataReader([company, trade_dic[company]],'yahoo', trade_start_date,trade_end_date)
        trade_close_price = trade_price['Close']
        trade_return_rate = trade_close_price.pct_change()
        x =trade_return_rate[company]
        X= sm.add_constant(x)
        y = trade_return_rate[trade_dic[company]]
        model = sm.OLS(y,X,missing='drop')
        result = model.fit()
        (a,b) = result.params
        trade_return_rate['Abnormal_Return'] = y - x * b - a
        trade_subevent_cumulative_abnormal_return[f'{company}'] = trade_return_rate['Abnormal_Return'].cumsum()[1:]
        trade_subevent_cumulative_abnormal_return
    
    trade_subevent_cumulative_abnormal_return['Average_CAR'] = trade_subevent_cumulative_abnormal_return.apply(lambda x: x.sum()/(len(trade_subevent_company_list)),axis = 1)
    trade_reg_result.append(trade_subevent_cumulative_abnormal_return['Average_CAR'])

# ### inputs for plotting
# num_trade_events = len(trade_eventdate_set) #8
# num_us_stock = len(trade_industry)/num_trade_events #30
# trade_event_list = list(trade_event_name[[0 + x*(len(trade_dataframe)-0)/len(trade_eventdate_set) for x in range(len(trade_eventdate_set))]])
# us_industry_list = list(trade_industry)[0:int(num_us_stock)]
# us_unique_industry_list = list(set(us_industry_list))
# us_unique_industry_count = list(np.unique(us_industry_list, return_counts=True)[1])
# us_stock_list = list(fed_company)[0:int(num_us_stock)]
# us_company_list = list(fed_dataframe['Company Name'])[0:int(num_us_stock)]

